
## what do I need to do?
- big map little map?
- 

# Exploration

In [321]:
!conda env list

[autoreload of clean_osm.cl failed: Traceback (most recent call last):
  File "/Users/josephmann/anaconda3/envs/fst/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/Users/josephmann/Documents/Notebooks/u2/clean_osm/cl.py", line 75
    !head map.osm
    ^
SyntaxError: invalid syntax
]


# conda environments:
#
fst                   *  /Users/josephmann/anaconda3/envs/fst
root                     /Users/josephmann/anaconda3



In [322]:
import sys

# sys.path.insert(0,'/Users/josephmann/Documents/GitHub/dask/dask/')

In [323]:
import numpy
import jmespath as jp
import pandas as pd
import re
from pymongo import MongoClient, GEO2D


In [324]:
from clean_osm import *
from importlib import reload

In [325]:
import clean_osm
reload(clean_osm)

<module 'clean_osm' from '/Users/josephmann/Documents/Notebooks/u2/clean_osm/__init__.py'>

In [326]:
reload(clean_osm.cl)

SyntaxError: invalid syntax (cl.py, line 75)

In [327]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [328]:
from clean_osm.cl import *

## Set up test suite

In [329]:
import pytest


In [330]:
# !rm sample*
pytest.main(['-v','clean_osm'])

================================================== test session starts ==================================================
platform darwin -- Python 3.6.1, pytest-3.1.2, py-1.4.34, pluggy-0.4.0 -- /Users/josephmann/anaconda3/envs/fst/bin/python
cachedir: .cache
rootdir: /Users/josephmann/Documents/Notebooks/u2, inifile:
collecting ... collected 5 items

clean_osm/test_clean.py::test_download_data PASSED
clean_osm/test_clean.py::test_get_element PASSED
clean_osm/test_clean.py::test_element2dict PASSED
clean_osm/test_clean.py::test_pf PASSED
clean_osm/test_clean.py::test_top_value_freqs PASSED

=============================================== 5 passed in 38.72 seconds ===============================================


0

## Create bag

In [331]:
b1 = cl.pf()

partition files still fresh


## tag frequencies

In [332]:
b1.npartitions

8

In [333]:
b1.take(1)

({'attr': {'changeset': '43379308',
   'id': '29811182',
   'lat': '53.5354110',
   'lon': '-113.5079960',
   'timestamp': '2016-11-03T14:01:28Z',
   'uid': '1943392',
   'user': 'LogicalViolinist',
   'version': '27'},
  'tag': {'admin_level': '8',
   'capital': '4',
   'ele': '645',
   'is_in': {'continent': 'North America',
    'country': 'Canada',
    'country_code': 'CA',
    'iso_3166_2': 'CA-AB',
    'root': 'Alberta; Canada',
    'state': 'Alberta',
    'state_code': 'AB'},
   'name': {'ar': 'إدمونتون',
    'de': 'Edmonton',
    'en': 'Edmonton',
    'fr': 'Edmonton',
    'root': 'Edmonton',
    'ru': 'Эдмонтон',
    'uk': 'Едмонтон'},
   'place': 'city',
   'population': '812201',
   'state_capital': 'yes',
   'wikidata': 'Q2096',
   'wikipedia': 'en:Edmonton'},
  'type': 'node'},)

In [334]:
b1.map(lambda d: jp.search('keys(@)[]', d)).flatten().frequencies().compute()

[('type', 422221), ('attr', 422221), ('tag', 422221)]

### `top_value_freqs`

In [335]:
def top_value_freqs(subtag, b1=b1, n=5):
    """
    parameters:
    ----------
    subtag (string): returns the top n frequencies of values
    for the key "subtag"
    if the value of "tag.[subtag]" is a dictionary then it returns 
    the top frequencies of the keys of  that dictionary.

    b1 (dask Bag): the bag that will be searched

    n (int): number of the most freqent tags and their frequencies to return

    returns:
    -------
    a dask bag of pairs (value, count)
    """

    expression = jp.compile(subtag)

    # make bag of subelements, remove None elements
    b2 = b1.map(lambda d: expression.search(d)).filter(lambda d: d)

    # make bag of lists comprised of key lists or just single element lists
    b3 = b2.map(lambda d: list(d.keys()) if isinstance(
        d, dict) else (d if isinstance(d, list) else [d]))

    # concatenate all lists together
    b4 = b3.flatten()

    # remove None elemens
    b5 = b4.filter(lambda d: d)

    # return bag with top n frequencies
    return b5.frequencies().topk(n, lambda x: x[1])

In [336]:
{k:w for k,w in cl.top_value_freqs('type', b1, n=20).compute()}

{'node': 366091, 'relation': 399, 'way': 55731}

In [337]:
top_value_freqs('attr.user', b1=b1, n=20).compute()

[('edmontongeo', 331406),
 ('Sundance', 13844),
 ('xixi', 10761),
 ('ionutr_telenav', 8807),
 ('Feldo', 8215),
 ('yegbin', 7611),
 ('Mesowhite', 7503),
 ('ij_', 4754),
 ('jdkrabbe', 4655),
 ('ptrchpmn', 3108),
 ('ec90', 1962),
 ('AKC', 1825),
 ('Viajero Perdido', 1758),
 ('xelloss', 1740),
 ('yoltia', 1400),
 ('flight780', 1203),
 ('alester', 1063),
 ('Ranek', 922),
 ('ntrussell', 917),
 ('spince', 603)]

In [338]:
pd.DataFrame.from_records(top_value_freqs('attr', b1, n=10).compute())

,0,1
0,changeset,422221
1,id,422221
2,timestamp,422221
3,uid,422221
4,user,422221
5,version,422221
6,lat,366091
7,lon,366091


In [339]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [340]:
%watermark

2017-08-05T17:43:42-06:00

CPython 3.6.1
IPython 5.4.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)
system     : Darwin
release    : 16.7.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [341]:
pd.DataFrame.from_records(top_value_freqs('type', b1, n=10).compute())

,0,1
0,node,366091
1,way,55731
2,relation,399


In [342]:
pd.DataFrame.from_records(top_value_freqs('tag', b1, n=10).compute())

,0,1
0,building,41773
1,source,21288
2,addr,19263
3,highway,7315
4,name,4884
5,service,1602
6,amenity,1525
7,lanes,1237
8,oneway,1130
9,landuse,986


In [343]:
pd.DataFrame.from_records(top_value_freqs('tag.is_in', b1).compute())

,0,1
0,"Edmonton,Alberta",401
1,Edmonton,7
2,continent,2
3,country,2
4,country_code,2


# Data Cleaning

## Tale of Two Cities

In [344]:
b1.map(
    lambda d: jp.search('tag.addr.city',d)).distinct().compute()


['Edmonton', 'edmonton', None]

In [345]:
top_value_freqs('tag.addr.city',b1).compute()

[('Edmonton', 13058), ('edmonton', 1)]

In [346]:

def fix_city(doc):
    path = 'tag.addr.city'
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        doc['tag']['addr']['city'] = 'Edmonton'

    return doc

In [347]:
top_value_freqs('addr.city', b1= b1.map(fix_city)).compute()

[]

## Five Provinces

In [348]:
b1.map(lambda d: jp.search('tag.addr.province', d)).filter(lambda d: d).distinct().compute()

['AB', 'AVB', 'Alberta;AB', 'Alberta', 'Aberta']

In [349]:
b1.map(lambda d: jp.search('tag.addr.state', d)).filter(lambda d: d).distinct().compute()

['AB']

### Provinces Fix

Previously we had the following distribution of entries for *province*.

In [350]:
pd.DataFrame(
    top_value_freqs('tag.addr.province',b1=b1).compute())

,0,1
0,AB,142
1,Alberta,29
2,AVB,1
3,Aberta,1
4,Alberta;AB,1


We change all entries with province to 'Alberta', independently of previous content.

In [351]:

def fix_province(doc):
    path = 'tag.addr.province'
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        doc['tag']['addr']['province'] = 'Alberta'

    return doc

In [352]:
b1.map(fix_province).map(lambda d: jp.search('tag.addr.province', d)).filter(
    lambda d: d).distinct().compute()

['Alberta']

In [353]:
pd.DataFrame(
    top_value_freqs('tag.addr.province', b1=b1.map(fix_province)).compute())

,0,1
0,Alberta,174


## Non-standard postal codes

In [354]:
postal_re = re.compile(r'[A-Z]\d[A-Z] \d[A-Z]\d')
b1.map(lambda d: jp.search('tag.addr.postcode', d)).filter(
    lambda d: d).filter(lambda c: not postal_re.match(c)).compute()

['T5J-1J4',
 'T6e 2g9',
 'T5L4G4',
 'T5C3C8',
 'T6E1Z2',
 'T6C1N8',
 'T5J1E3',
 'AB T5J',
 'T5J1A7',
 'Alberta T6G',
 't6e 2g9',
 'AB T6E4S6',
 'T5J4B7',
 'T5H3J7',
 'T5K2X4',
 'T5N2N8',
 'T6G1C9',
 'T6E',
 'T5M0J2',
 'T5K0G1',
 'T5H2Z4',
 'T5H1A8',
 'T6E3Z5',
 'T6E3Z4',
 'T5M2P2',
 'T5J-0R2',
 'T5M0S7',
 'T6E',
 'T5N2T3',
 'T5R0L9',
 'T5N3L5',
 'T5N1H6',
 'T6A3X4',
 'T5N0N7',
 'T5N2V3',
 'T6C1W5',
 'T5M0V4',
 'T5H0B1',
 'T5G2E7',
 'T5B0A1',
 'T5N0W4',
 'T6G1T5',
 'T5M1P8',
 'T5J',
 'T5J5A2',
 'T5G0A6',
 'T6G',
 'T6G',
 'T5J',
 'T5J',
 'T5G3K2',
 'T5H2L3',
 'T6C1B5',
 'T5J',
 'T6G0X4',
 'T5J',
 'T5J',
 'T5H',
 'T5K2V4',
 'T5R5B3',
 'T6C1N8',
 'T6A1B6']

### Fix postal codes

In [355]:

def fix_postal_codes(doc):
    path = 'tag.addr.postcode'
    pc_re = re.compile(r'([A-Z]\d[A-Z])[-\s]*(\d[A-Z]\d){0,1}', re.IGNORECASE)
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        matches = pc_re.search(value)
        if matches:
            s = "{}".format(matches.group(1))
            if matches.group(2):
                s += " " + matches.group(2)
            s = s.upper()
            doc['tag']['addr']['postcode'] = s

    return doc

In [356]:
postal_re = re.compile(r'[A-Z]\d[A-Z]( \d[A-Z]\d){0,1}')
b1.map(fix_postal_codes).map(lambda d: jp.search('tag.addr.postcode', d)).filter(
    lambda d: d).filter(lambda c: not postal_re.match(c)).compute()

[]

## Create point structure for mongo to use `$geonear`

In [390]:
def create_point(doc):
    """
        doc should contain subdoc attr.lon and attr.lat, 
        if it doesn't then we don't create the point structure
    """
    if 'attr' in doc:
        if ('lon' in doc['attr']) and ('lat' in doc['attr']):
            lon = float(doc['attr']['lon'] )
            lat = float(doc['attr']['lat'] )
            doc['attr']['point'] = {'type' : 'Point', 'coordinates' : [lon, lat]}
#         else:
#             doc['attr']['point'] = {'coordinates': [np.nan, np.nan]}
#     else:
#         doc['attr'] = {'point': {'coordinates': [np.nan, np.nan]}}
    return doc

In [391]:
b1.map(create_point).take(3)

({'attr': {'changeset': '43379308',
   'id': '29811182',
   'lat': '53.5354110',
   'lon': '-113.5079960',
   'point': {'coordinates': [-113.507996, 53.535411], 'type': 'Point'},
   'timestamp': '2016-11-03T14:01:28Z',
   'uid': '1943392',
   'user': 'LogicalViolinist',
   'version': '27'},
  'tag': {'admin_level': '8',
   'capital': '4',
   'ele': '645',
   'is_in': {'continent': 'North America',
    'country': 'Canada',
    'country_code': 'CA',
    'iso_3166_2': 'CA-AB',
    'root': 'Alberta; Canada',
    'state': 'Alberta',
    'state_code': 'AB'},
   'name': {'ar': 'إدمونتون',
    'de': 'Edmonton',
    'en': 'Edmonton',
    'fr': 'Edmonton',
    'root': 'Edmonton',
    'ru': 'Эдмонтон',
    'uk': 'Едмонтон'},
   'place': 'city',
   'population': '812201',
   'state_capital': 'yes',
   'wikidata': 'Q2096',
   'wikipedia': 'en:Edmonton'},
  'type': 'node'},
 {'attr': {'changeset': '39441015',
   'id': '30118680',
   'lat': '53.5266842',
   'lon': '-113.4945885',
   'point': {'coordi

In [359]:
import numpy as np

In [392]:
b1.map(create_point).filter(lambda d: 'point' not in d['attr']).compute()

[{'attr': {'changeset': '46245638',
   'id': '4734663',
   'timestamp': '2017-02-20T14:32:24Z',
   'uid': '4930407',
   'user': 'ionutr_telenav',
   'version': '55'},
  'tag': {'highway': 'secondary',
   'lanes': '2',
   'lit': 'yes',
   'name': '109 Street NW',
   'oneway': 'yes'},
  'type': 'way'},
 {'attr': {'changeset': '49049098',
   'id': '4734665',
   'timestamp': '2017-05-28T16:18:29Z',
   'uid': '6085917',
   'user': 'Zan Thrax',
   'version': '68'},
  'tag': {'highway': 'secondary',
   'lanes': '2',
   'maxspeed': '50',
   'name': '104 Street NW',
   'oneway': 'yes',
   'surface': 'asphalt'},
  'type': 'way'},
 {'attr': {'changeset': '34237437',
   'id': '4734674',
   'timestamp': '2015-09-25T06:07:12Z',
   'uid': '2525040',
   'user': 'flight780',
   'version': '29'},
  'tag': {'access': 'designated',
   'highway': 'service',
   'name': '114 Street NW',
   'oneway': 'yes'},
  'type': 'way'},
 {'attr': {'changeset': '33707102',
   'id': '4734677',
   'timestamp': '2015-08-31T

In [400]:
b1.map(create_point).map(lambda d: jp.search('attr.point', d)).take(10)
# b1.map(create_point).filter(lambda d: d.get('attr.point')).pluck('attr').pluck('point').compute()

({'coordinates': [-113.507996, 53.535411], 'type': 'Point'},
 {'coordinates': [-113.4945885, 53.5266842], 'type': 'Point'},
 {'coordinates': [-113.2968767, 53.6134201], 'type': 'Point'},
 {'coordinates': [-113.6423221, 53.4368988], 'type': 'Point'},
 {'coordinates': [-113.6252971, 53.4792229], 'type': 'Point'},
 {'coordinates': [-113.2942009, 53.6405376], 'type': 'Point'},
 {'coordinates': [-113.4685521, 53.5519448], 'type': 'Point'},
 {'coordinates': [-113.6568045, 53.4300519], 'type': 'Point'},
 {'coordinates': [-113.4896364, 53.5298217], 'type': 'Point'},
 {'coordinates': [-113.5880262, 53.4848878], 'type': 'Point'})

In [406]:
b1.map(create_point).map(lambda d: jp.search('attr.point.coordinates[0]', d)).filter(lambda d: d).min().compute()

-113.6586842

In [363]:
b1.map(create_point).pluck('attr').pluck('point').pluck('coordinates').pluck(1).min().compute()

53.3956445

In [364]:
b1.map(create_point).pluck('attr').pluck('point').pluck('coordinates').pluck(0).max().compute()

-112.0370555

In [365]:
b1.map(create_point).pluck('attr').pluck('point').pluck('coordinates').pluck(0).min().compute()

-113.6586842

## Create bag of json strings and write to disk

In [366]:
clean_b = b1.map(fix_postal_codes).map(fix_city).map(fix_province).map(create_point).map(json.dumps)

In [367]:
clean_b.take(1)

('{"type": "node", "attr": {"changeset": "43379308", "id": "29811182", "lat": "53.5354110", "lon": "-113.5079960", "timestamp": "2016-11-03T14:01:28Z", "uid": "1943392", "user": "LogicalViolinist", "version": "27", "point": {"type": "Point", "coordinates": [-113.507996, 53.535411]}}, "tag": {"admin_level": "8", "capital": "4", "ele": "645", "is_in": {"root": "Alberta; Canada", "continent": "North America", "country": "Canada", "country_code": "CA", "iso_3166_2": "CA-AB", "state": "Alberta", "state_code": "AB"}, "name": {"root": "Edmonton", "ar": "\\u0625\\u062f\\u0645\\u0648\\u0646\\u062a\\u0648\\u0646", "de": "Edmonton", "en": "Edmonton", "fr": "Edmonton", "ru": "\\u042d\\u0434\\u043c\\u043e\\u043d\\u0442\\u043e\\u043d", "uk": "\\u0415\\u0434\\u043c\\u043e\\u043d\\u0442\\u043e\\u043d"}, "place": "city", "population": "812201", "state_capital": "yes", "wikidata": "Q2096", "wikipedia": "en:Edmonton"}}',)

In [368]:
# write bag to disk
clean_b.to_textfiles('clean-*.json')

In [369]:
!ls -lh clean-*.json

-rw-r--r--@ 1 josephmann  staff    15M  5 Aug 17:45 clean-0.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-1.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-2.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-3.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-4.json
-rw-r--r--@ 1 josephmann  staff    17M  5 Aug 17:45 clean-5.json
-rw-r--r--@ 1 josephmann  staff   7.0M  5 Aug 17:45 clean-6.json
-rw-r--r--@ 1 josephmann  staff   6.4M  5 Aug 17:45 clean-7.json


## Amenity frequencies

In [370]:
top_value_freqs('tag.amenity', n=10).compute()

[('parking', 521),
 ('restaurant', 168),
 ('fast_food', 129),
 ('cafe', 84),
 ('bicycle_parking', 64),
 ('bench', 60),
 ('school', 54),
 ('bank', 36),
 ('fuel', 36),
 ('place_of_worship', 36)]

In [371]:
b1.filter(lambda d: jp.search(
        "tag.amenity",d) == 'restaurant').pluck(
    'tag').pluck(
    'name','Nameless').filter(lambda d: isinstance(d, dict)).compute()

[{'en': 'Viphalay', 'root': 'Vidaley'},
 {'en': 'Maxs', 'root': 'Maxs'},
 {'fr': '爱民顿', 'root': '爱民顿'}]

In [372]:
b1.filter(lambda d: jp.search("tag.amenity", d) == 'restaurant').pluck(
    'tag', 'NoTag').pluck(
    'name', 'Nameless').map(lambda d: d.get('root', 'NoRoot') if isinstance(d, dict) else d).frequencies().topk(10, lambda t: t[1]).compute()

[('Nameless', 8),
 ('Boston Pizza', 5),
 ("Ricky's All Day Grill", 2),
 ("Tony Roma's", 2),
 ('Urban Diner', 2),
 ("Ruth's Chris Steakhouse", 2),
 ('Chop Steakhouse', 2),
 ('Urban China', 1),
 ("Humpty's Family Restaurant", 1),
 ("Moxie's", 1)]

In [373]:
b1.filter(lambda d: jp.search("tag.amenity",d) == 'restaurant').pluck(
    'tag',default='NoTag').map(lambda d: d.get('name', "__No_Name__")).compute()

['Urban China',
 "Humpty's Family Restaurant",
 "Moxie's",
 "Ricky's All Day Grill",
 "Chandler's The Tea Shoppe",
 "Kamil's Turkish Cafe & Restaurant",
 'Lee Garden',
 "Tony Roma's",
 'Nha Trang Vietnamese Restaurant',
 'Pho Hoan Pasteur Fusion Noodle House',
 'Blue Plate Diner',
 'Mikado',
 'Delux Burger Bar',
 "That's Aroma! A Garlic Restaurant",
 "Vi's for Pies",
 'C&S Seafood Restaurant',
 'Century Palace Restaurant',
 'Double Happiness',
 'Mirama Dining & Lounge',
 'El Rancho',
 'Trang Tien',
 'Mucho Burrito',
 'High Level Diner',
 'RedBike',
 'All Happy Family Restaurant',
 'Kyoto Japanese Cuisine',
 'Phơbulous',
 'Royal Pizza',
 'Pagolac Restaurant',
 "Doan's",
 'Culina Restaurant',
 "B's Diner",
 'Upper Crust Cafe & Caterers',
 'New Asian Village',
 "Bird's Nest of Beijing Restaurant",
 "Denny's",
 "Joey Tomato's",
 'Old Szechuan Restaurant',
 'Vighalay Thai Laos',
 'The Wokkery Restaurant',
 "Uncle Albert's Family Restaurant",
 'Noodle Noodle Restaurant',
 'Coliseum Steak Rib

In [374]:
b1.filter(lambda d: jp.search("tag.amenity",d) == 'restaurant').pluck(
    'tag',default='NoTag').pluck(
    'name','__No_Name__').map(lambda d: d.get('root','__No_Root__') if isinstance(d, dict) else d).frequencies().topk(10, lambda t: t[1]).compute()

[('__No_Name__', 8),
 ('Boston Pizza', 5),
 ("Ricky's All Day Grill", 2),
 ("Tony Roma's", 2),
 ('Urban Diner', 2),
 ("Ruth's Chris Steakhouse", 2),
 ('Chop Steakhouse', 2),
 ('Urban China', 1),
 ("Humpty's Family Restaurant", 1),
 ("Moxie's", 1)]

In [375]:
b1.filter(lambda d: jp.search(
        "tag.amenity",d) == 'cafe').pluck(
    'tag').pluck(
    'name','__No_Name__').map(lambda d: d['root'] if isinstance(d, dict) else d).frequencies().topk(10,lambda x:x[1]).compute()

[('Second Cup', 10),
 ('Starbucks Coffee', 9),
 ('Starbucks', 6),
 ('Tim Hortons', 5),
 ('__No_Name__', 4),
 ('Transcend Coffee', 2),
 ('Orange Julius', 2),
 ('Good Earth', 2),
 ("Jeffrey's Café & Wine Bar", 1),
 ('The Second Cup', 1)]

# Mongodb 

In [376]:
!ls -lh clean-?.json


-rw-r--r--@ 1 josephmann  staff    15M  5 Aug 17:45 clean-0.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-1.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-2.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-3.json
-rw-r--r--@ 1 josephmann  staff    18M  5 Aug 17:45 clean-4.json
-rw-r--r--@ 1 josephmann  staff    17M  5 Aug 17:45 clean-5.json
-rw-r--r--@ 1 josephmann  staff   7.0M  5 Aug 17:45 clean-6.json
-rw-r--r--@ 1 josephmann  staff   6.4M  5 Aug 17:45 clean-7.json


In [377]:
client = MongoClient('localhost:27017')
mdb = client['examples']

In [378]:
mdb.p33.drop()

In [379]:
%%bash
ls clean-?.json | while read col; do
    mongoimport --db examples --collection p33 -j 8 --type json --file $col;
done

2017-08-05T17:46:11.376-0600	connected to: localhost
2017-08-05T17:46:12.760-0600	imported 49671 documents
2017-08-05T17:46:12.782-0600	connected to: localhost
2017-08-05T17:46:14.001-0600	imported 63131 documents
2017-08-05T17:46:14.021-0600	connected to: localhost
2017-08-05T17:46:15.352-0600	imported 62959 documents
2017-08-05T17:46:15.377-0600	connected to: localhost
2017-08-05T17:46:16.716-0600	imported 62938 documents
2017-08-05T17:46:16.739-0600	connected to: localhost
2017-08-05T17:46:17.970-0600	imported 62946 documents
2017-08-05T17:46:17.993-0600	connected to: localhost
2017-08-05T17:46:19.232-0600	imported 62906 documents
2017-08-05T17:46:19.255-0600	connected to: localhost
2017-08-05T17:46:19.799-0600	imported 29220 documents
2017-08-05T17:46:19.821-0600	connected to: localhost
2017-08-05T17:46:20.347-0600	imported 28450 documents


In [380]:
mdb.p33.count({})

422221

In [381]:
# list(mdb.p33.find({"tag.landuse": {"$exists": 1}},{"tag.landuse": 1,'_id':0}))


In [382]:
list(
    mdb.p33.aggregate([{
        "$sortByCount": "$tag.landuse"
    }, {
        "$limit": 30
    }]))

[{'_id': None, 'count': 421235},
 {'_id': 'residential', 'count': 560},
 {'_id': 'grass', 'count': 179},
 {'_id': 'retail', 'count': 127},
 {'_id': 'commercial', 'count': 59},
 {'_id': 'forest', 'count': 14},
 {'_id': 'recreation_ground', 'count': 14},
 {'_id': 'construction', 'count': 9},
 {'_id': 'industrial', 'count': 4},
 {'_id': 'cemetery', 'count': 3},
 {'_id': 'basin', 'count': 3},
 {'_id': 'reservoir', 'count': 3},
 {'_id': 'traffic_island', 'count': 2},
 {'_id': 'military', 'count': 2},
 {'_id': 'allotments', 'count': 2},
 {'_id': 'public', 'count': 1},
 {'_id': 'brownfield', 'count': 1},
 {'_id': 'mixed', 'count': 1},
 {'_id': 'publicllotmentsllotments', 'count': 1},
 {'_id': 'quarry', 'count': 1}]

In [383]:
list(
    mdb.p33.aggregate([{
        "$sortByCount": { "$substrBytes": ["$tag.addr.postcode", 0, 3]}
    }, {
        "$limit": 30
    }]))

[{'_id': '', 'count': 421869},
 {'_id': 'T5J', 'count': 95},
 {'_id': 'T6E', 'count': 46},
 {'_id': 'T6G', 'count': 41},
 {'_id': 'T5M', 'count': 40},
 {'_id': 'T5N', 'count': 40},
 {'_id': 'T6C', 'count': 24},
 {'_id': 'T5H', 'count': 21},
 {'_id': 'T5K', 'count': 20},
 {'_id': 'T5G', 'count': 11},
 {'_id': 'T6A', 'count': 4},
 {'_id': 'T5R', 'count': 3},
 {'_id': 'T5B', 'count': 2},
 {'_id': 'T6Z', 'count': 1},
 {'_id': 'T6J', 'count': 1},
 {'_id': 'T5C', 'count': 1},
 {'_id': 'T5L', 'count': 1},
 {'_id': 'T6K', 'count': 1}]

In [384]:
list(
    mdb.p33.aggregate([{
        "$match": {
            "tag.landuse": {
                "$in": ["grass", "forest", "recreation_ground"]
            }
        }
    }, {
        "$sortByCount": "$tag"
    }, {
        "$limit": 30
    }]))

[{'_id': {'landuse': 'grass'}, 'count': 72},
 {'_id': {'golf': 'fairway', 'landuse': 'grass'}, 'count': 39},
 {'_id': {'golf': 'tee', 'landuse': 'grass'}, 'count': 18},
 {'_id': {'landuse': 'grass', 'type': 'multipolygon'}, 'count': 8},
 {'_id': {'landuse': 'forest', 'leaf_cycle': 'mixed', 'leaf_type': 'mixed'},
  'count': 7},
 {'_id': {'landuse': 'recreation_ground'}, 'count': 6},
 {'_id': {'landuse': 'forest'}, 'count': 4},
 {'_id': {'area': 'yes', 'landuse': 'grass'}, 'count': 3},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '17',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '1',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '6',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '12',
   'sport': 'golf'},
  'count': 2},
 {'_id': 

In [385]:
list(
    mdb.p33.aggregate([ {"$match": {"tag.landuse": "grass"}},  
                       {"$sortByCount" : "$attr.user"},
        {"$limit": 30}
    ]))

[{'_id': 'jdkrabbe', 'count': 95},
 {'_id': 'yegbin', 'count': 60},
 {'_id': 'yoltia', 'count': 12},
 {'_id': 'Mesowhite', 'count': 7},
 {'_id': 'Moham837', 'count': 4},
 {'_id': 'Maarten Deen', 'count': 1}]

In [386]:
# how do I get the number of green spaces sorted by postal code?

r =     mdb.p33.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"user": "$attr.user","postcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postcode": "$_id.postcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 20}
    ])

_df = pd.DataFrame.from_records(r)

In [387]:
_df

,count,postcode,user
0,62,T5J,Mesowhite
1,27,T5M,ptrchpmn
2,17,T6E,yegbin
3,13,T6C,yegbin
4,11,T5N,ptrchpmn
5,11,T5N,edmontongeo
6,11,T6E,Mesowhite
7,10,T6G,yegbin
8,10,T6G,Mesowhite
9,10,T5J,Yottabyte


In [388]:
_s = _df.loc[:,['count']].sum().fillna('')
_s.name = "col_sum"

_df.append( _s )

,count,postcode,user
0,62.0,T5J,Mesowhite
1,27.0,T5M,ptrchpmn
2,17.0,T6E,yegbin
3,13.0,T6C,yegbin
4,11.0,T5N,ptrchpmn
5,11.0,T5N,edmontongeo
6,11.0,T6E,Mesowhite
7,10.0,T6G,yegbin
8,10.0,T6G,Mesowhite
9,10.0,T5J,Yottabyte


## mongodb indexing and geonear

## problem attr.point.coordinates should either have valid data or not exist...

In [389]:
# , GEO2D
# mdb.p33.create_index( ( 'attr.point' , "2dsphere")  )
mdb.p33.create_index([( 'attr.point.coordinates', GEO2D)])
# db.places.createIndex( { loc : "2dsphere" } )

OperationFailure: point not in interval of [ -180, 180 ] :: caused by :: { _id: ObjectId('598658cb23d1bdcb6b1b320a'), type: "way", attr: { changeset: "14482624", id: "4744100", timestamp: "2012-12-31T22:40:10Z", uid: "239998", user: "Sundance", version: "24", point: { coordinates: [ nan.0, nan.0 ] } }, tag: { highway: "secondary", junction: "roundabout", lanes: "2", oneway: "yes" } }

In [314]:
import pprint
for d in mdb.p33.find({"attr.point.coordinates" : {"$near" : [53.5, -133.5] }}).limit(4):
    pprint.pprint(d)

{'_id': ObjectId('597e184423d1bdcb6b13e924'),
 'attr': {'changeset': '37712677',
          'id': '4051253340',
          'lat': '53.9465480',
          'lon': '-112.0370555',
          'point': {'coordinates': [-112.0370555, 53.946548], 'type': 'Point'},
          'timestamp': '2016-03-09T16:31:26Z',
          'uid': '307202',
          'user': 'alester',
          'version': '1'},
 'tag': {},
 'type': 'node'}
{'_id': ObjectId('597e184423d1bdcb6b13db84'),
 'attr': {'changeset': '37712677',
          'id': '4051253354',
          'lat': '53.9466598',
          'lon': '-112.0382955',
          'point': {'coordinates': [-112.0382955, 53.9466598], 'type': 'Point'},
          'timestamp': '2016-03-09T16:31:26Z',
          'uid': '307202',
          'user': 'alester',
          'version': '1'},
 'tag': {},
 'type': 'node'}
{'_id': ObjectId('597e184423d1bdcb6b13e2e1'),
 'attr': {'changeset': '37712677',
          'id': '4051253374',
          'lat': '53.9469388',
          'lon': '-112.039613

In [320]:
import pprint
from bson.son import SON

# this returns the postal code of the closest node that has a postal code

for d in mdb.p33.find({
        "attr.point.coordinates" : SON([("$near" , [53.5, -113.5]), ("$maxDistance", 300) ]),
        "tag.addr.postcode": {"$exists" : 1}},
        {"tag.addr.postcode": 1, "_id" : 0 }
    ).limit(1):
    pprint.pprint(d)

{'tag': {'addr': {'postcode': 'T6C 0Y4'}}}


# Big(ger) Data

In [68]:
# min_lat, max_lat, min_lon, max_lon =  53.5164, 53.5718, -113.5742,-113.4485
min_lat=53.4271
min_lon=-113.666
max_lat=53.6495
max_lon=-113.34

map_url = 'http://overpass-api.de/api/map?bbox={0},{1},{2},{3}'.format(min_lon, min_lat, max_lon, max_lat)
cl.download_data(map_url, fn='bigmap.osm')

In [69]:
import os
os.path.getsize('bigmap.osm') / 1024**2 , os.path.getsize('map.osm') / 1024**2

(502.4631099700928, 79.05549621582031)

In [70]:
!ls -lh *map.osm 

-rw-r--r--@ 1 josephmann  staff   502M  6 Jun 17:23 bigmap.osm
-rw-r--r--@ 1 josephmann  staff    79M 23 Jun 15:53 map.osm


In [71]:
# create partitions for parallel processing
big_b = cl.pf(fn = 'bigmap.osm')

partition files still fresh


In [72]:
!ls *_partition_?.osm

bigmap_partition_0.osm bigmap_partition_6.osm map_partition_4.osm
bigmap_partition_1.osm bigmap_partition_7.osm map_partition_5.osm
bigmap_partition_2.osm map_partition_0.osm    map_partition_6.osm
bigmap_partition_3.osm map_partition_1.osm    map_partition_7.osm
bigmap_partition_4.osm map_partition_2.osm
bigmap_partition_5.osm map_partition_3.osm


Clean up our bag of data with 3 funcs

In [73]:
clean_b = big_b.map(fix_postal_codes).map(fix_city).map(fix_province).map(json.dumps)

In [74]:
top_value_freqs('attr.user', b1=b1, n=20).compute()

[('edmontongeo', 331406),
 ('Sundance', 13844),
 ('xixi', 10761),
 ('ionutr_telenav', 8807),
 ('Feldo', 8215),
 ('yegbin', 7611),
 ('Mesowhite', 7503),
 ('ij_', 4754),
 ('jdkrabbe', 4655),
 ('ptrchpmn', 3108),
 ('ec90', 1962),
 ('AKC', 1825),
 ('Viajero Perdido', 1758),
 ('xelloss', 1740),
 ('yoltia', 1400),
 ('flight780', 1203),
 ('alester', 1063),
 ('Ranek', 922),
 ('ntrussell', 917),
 ('spince', 603)]

In [75]:
top_value_freqs('attr.user', b1=big_b, n=20).compute()

[('edmontongeo', 2142198),
 ('xixi', 112465),
 ('Mesowhite', 94015),
 ('Sundance', 73349),
 ('yegbin', 22367),
 ('Shawn Benbow', 21194),
 ('charrois', 15168),
 ('bogdanp_telenav', 14462),
 ('Oberaffe', 13987),
 ('xelloss', 13415),
 ('geobase_stevens', 11148),
 ('Feldo', 9753),
 ('Viajero Perdido', 9322),
 ('ionutr_telenav', 8525),
 ('ec90', 8097),
 ('GiJo', 7861),
 ('jdkrabbe', 5672),
 ('VE6SRV', 5602),
 ('ntrussell', 5518),
 ('ij_', 5201)]

Write clean bag to json files so they can subsequently be imported into mongo database

In [76]:
clean_b.to_textfiles('big_sample-*.json')

In [77]:
!ls -lh big_sample*

-rw-r--r--@ 1 josephmann  staff    76M 29 Jul 12:50 big_sample-0.json
-rw-r--r--@ 1 josephmann  staff    84M 29 Jul 12:50 big_sample-1.json
-rw-r--r--@ 1 josephmann  staff    84M 29 Jul 12:50 big_sample-2.json
-rw-r--r--@ 1 josephmann  staff    84M 29 Jul 12:50 big_sample-3.json
-rw-r--r--@ 1 josephmann  staff    84M 29 Jul 12:50 big_sample-4.json
-rw-r--r--@ 1 josephmann  staff    84M 29 Jul 12:50 big_sample-5.json
-rw-r--r--@ 1 josephmann  staff    37M 29 Jul 12:49 big_sample-6.json
-rw-r--r--@ 1 josephmann  staff    35M 29 Jul 12:49 big_sample-7.json


In [78]:
# not necessary because we're making another collection "big"
# mdb.p33.drop()

In [79]:
%%bash
ls big_sample-?.json | while read col; do
    mongoimport --db examples --collection big -j 8 --type json --file $col;
done

2017-07-29T12:50:20.773-0600	connected to: localhost
2017-07-29T12:50:23.772-0600	[###############.........] examples.big	49.9MB/75.9MB (65.7%)
2017-07-29T12:50:25.366-0600	[########################] examples.big	75.9MB/75.9MB (100.0%)
2017-07-29T12:50:25.366-0600	imported 325820 documents
2017-07-29T12:50:25.387-0600	connected to: localhost
2017-07-29T12:50:28.384-0600	[##############..........] examples.big	49.5MB/84.3MB (58.7%)
2017-07-29T12:50:30.485-0600	[########################] examples.big	84.3MB/84.3MB (100.0%)
2017-07-29T12:50:30.485-0600	imported 400272 documents
2017-07-29T12:50:30.507-0600	connected to: localhost
2017-07-29T12:50:33.504-0600	[#############...........] examples.big	48.6MB/84.3MB (57.6%)
2017-07-29T12:50:35.786-0600	[########################] examples.big	84.3MB/84.3MB (100.0%)
2017-07-29T12:50:35.786-0600	imported 400536 documents
2017-07-29T12:50:35.814-0600	connected to: localhost
2017-07-29T12:50:38.814-0600	[##############..........] examples.big	49.3M

In [120]:
mdb.big.count({}),mdb.big.count({'type': 'node'}),mdb.big.count({'type': 'way'})

(5384300, 4683712, 696158)

In [81]:
list(mdb.big.aggregate([{"$sortByCount":'$type'}]))

[{'_id': 'node', 'count': 4683712},
 {'_id': 'way', 'count': 696158},
 {'_id': 'relation', 'count': 4430}]

In [123]:
# how do I get the number of green spaces sorted by postal code?
list(
    mdb.p33.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"user": "$attr.user","postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postalcode": "$_id.postalcode", "count":1, "_id":0}},
#         # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 40}
    ]))

[{'count': 63, 'postalcode': 'T5J', 'user': 'Mesowhite'},
 {'count': 27, 'postalcode': 'T5M', 'user': 'ptrchpmn'},
 {'count': 17, 'postalcode': 'T6E', 'user': 'yegbin'},
 {'count': 13, 'postalcode': 'T6C', 'user': 'yegbin'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'ptrchpmn'},
 {'count': 11, 'postalcode': 'T6E', 'user': 'Mesowhite'},
 {'count': 11, 'postalcode': 'T5J', 'user': 'Yottabyte'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'edmontongeo'},
 {'count': 10, 'postalcode': 'T6G', 'user': 'yegbin'},
 {'count': 10, 'postalcode': 'T6G', 'user': 'Mesowhite'},
 {'count': 9, 'postalcode': 'T5K', 'user': 'Mesowhite'},
 {'count': 9, 'postalcode': 'T5N', 'user': 'TristanA'},
 {'count': 7, 'postalcode': 'T5M', 'user': 'edmontongeo'},
 {'count': 6, 'postalcode': 'T6G', 'user': 'ptrchpmn'},
 {'count': 5, 'postalcode': 'T6C', 'user': 'Mesowhite'},
 {'count': 5, 'postalcode': 'T5H', 'user': 'Mesowhite'},
 {'count': 4, 'postalcode': 'T6G', 'user': 'Yottabyte'},
 {'count': 4, 'postalcode': 'T

There are very few green spaces with postal code attributes, what to do?

In [147]:
list(
    mdb.big.aggregate([
        {"$match": {
            "tag.landuse": {
                "$in": ["grass", "forest", "recreation_ground"]
            }
        }},
    
            # group on user and postcode
        {"$group" : 
            {
        #         "_id" : {"postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}},
                "_id" : {"usage" : "$tag.landuse", "postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}},
                "count" : {"$sum" : 1}
            }
        },
#         {
#             "$sortByCount": "$tag"
#         }, 
        {
            "$limit": 30
        }
        ]))

[{'_id': {'postalcode': 'T6X', 'usage': 'recreation_ground'}, 'count': 2},
 {'_id': {'postalcode': '', 'usage': 'forest'}, 'count': 98},
 {'_id': {'postalcode': '', 'usage': 'grass'}, 'count': 702},
 {'_id': {'postalcode': '', 'usage': 'recreation_ground'}, 'count': 98}]

In [166]:
_c = mdb.big.find({'type':'node'}).limit(5)

In [167]:
_c.count()

4683712

We believe that every `node` has a _longitude_ and _latitude_.

In [176]:
_c = mdb.big.find({'type':'node', 'attr.lon': {'$exists': True}})

In [177]:
_c.count()

4683712

In [179]:
# per record updating requires, looping on a cursor: eg
### update of Second Cup
# this looks like it would be _slooowww_
for d in mdb.p33.find({"properties.name": {"$regex" : "Second Cup"}},{'_id':1}):
    mdb.p33.update_one({"_id": d["_id"]}, {"$set" :{"properties.name": 'Second Cup'}})

# mdb.p33.update(
#     {'type' : 'node'},
#     {'$set' : {
#         'attr.location': {}}}
# )

In [183]:
# per record updating requires, looping on a cursor: eg
### update of Second Cup
# this looks like it would be _slooowww_ - and it was.
for d in mdb.p33.find({"type": "node"},{'_id':1,'attr':1}):
    mdb.p33.update_one({"_id": d["_id"]}, {"$set" :{"attr.location": [ d['attr']['lon'], d['attr']['lat']]}})

# mdb.p33.update(
#     {'type' : 'node'},
#     {'$set' : {
#         'attr.location': {}}}
# )

In [185]:
mdb.p33.find_one({})

{'_id': ObjectId('597ce82523d1bdcb6b025536'),
 'attr': {'changeset': '39441015',
  'id': '30118680',
  'lat': '53.5266842',
  'location': ['-113.4945885', '53.5266842'],
  'lon': '-113.4945885',
  'timestamp': '2016-05-20T03:45:53Z',
  'uid': '2365330',
  'user': 'Viajero Perdido',
  'version': '6'},
 'tag': {},
 'type': 'node'}

In [ ]:
#using dask


In [ ]:
# how do I get the number of green spaces sorted by postal code?
list(
    mdb.big.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}},
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postalcode": "$_id.postalcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 40}
    ]))

In [86]:
# how do I get the number of green spaces sorted by postal code?
list(
    mdb.big.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"postalcode": "$_id.postalcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 40}
    ]))

[{'count': 274, 'postalcode': 'T6T'},
 {'count': 200, 'postalcode': 'T5J'},
 {'count': 184, 'postalcode': 'T6E'},
 {'count': 170, 'postalcode': 'T6L'},
 {'count': 126, 'postalcode': 'T5T'},
 {'count': 100, 'postalcode': 'T6G'},
 {'count': 92, 'postalcode': 'T6K'},
 {'count': 88, 'postalcode': 'T5M'},
 {'count': 82, 'postalcode': 'T6B'},
 {'count': 80, 'postalcode': 'T5N'},
 {'count': 62, 'postalcode': 'T6H'},
 {'count': 58, 'postalcode': 'T6C'},
 {'count': 54, 'postalcode': 'T6A'},
 {'count': 48, 'postalcode': 'T6J'},
 {'count': 46, 'postalcode': 'T5A'},
 {'count': 46, 'postalcode': 'T5E'},
 {'count': 46, 'postalcode': 'T5H'},
 {'count': 46, 'postalcode': 'T5P'},
 {'count': 44, 'postalcode': 'T5K'},
 {'count': 42, 'postalcode': 'T5R'},
 {'count': 42, 'postalcode': 'T5X'},
 {'count': 38, 'postalcode': 'T5Z'},
 {'count': 32, 'postalcode': 'T5C'},
 {'count': 32, 'postalcode': 'T5W'},
 {'count': 32, 'postalcode': 'T5S'},
 {'count': 32, 'postalcode': 'T5L'},
 {'count': 26, 'postalcode': 'T5